In [397]:
from transformers import ViTConfig, ViTModel 
from argparse import ArgumentParser
import numpy as np
import os
import pytorch_lightning as pl
from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning import Callback, seed_everything
from torch.utils.data.sampler import SubsetRandomSampler
import torch
from torch import nn
import torchvision
from torchvision import transforms as T

from pytorch_lightning.loggers import WandbLogger

In [398]:
#####################################
# constants
# TODO: these might need to be updated overtime ...
known_exit_rt = [3.5720, 4.9740, 7.0156, 11.6010, 27.5720]
unknown_exit_rt = [4.2550, 5.9220, 8.2368, 13.0090, 28.1661]

known_thresholds = [0.0035834426525980234, 0.0035834424197673798,
                    0.0035834426525980234, 0.0035834424197673798, 0.0035834424197673798]
unknown_thresholds = [0.0035834426525980234, 0.0035834424197673798,
                      0.0035834426525980234, 0.0035834424197673798, 0.0035834424197673798]
#                                                       
#####################################

In [399]:
# add your psych dataset right here ...

from torch.utils.data import Dataset
import os
import pandas as pd
import random
# from skimage import io

from PIL import Image, ImageFilter

from torchvision.utils import save_image

class OmniglotReactionTimeDataset(Dataset):
    """
    Dataset for omniglot + reaction time data
    Dasaset Structure:
    label1, label2, real_file, generated_file, reaction time
    ...
    args:
    - path: string - path to dataset (should be a csv file)
    - transforms: torchvision.transforms - transforms on the data
    """

    def __init__(self, data_file, transforms=None):
        self.raw_data = pd.read_csv(data_file)
        self.transform = transforms

    def __len__(self):
        return len(self.raw_data)

    def __getitem__(self, idx):
        label1 = int(self.raw_data.iloc[idx, 0])
        label2 = int(self.raw_data.iloc[idx, 1])

        im1name = self.raw_data.iloc[idx, 2]
        image1 = Image.open(im1name)
        im2name = self.raw_data.iloc[idx, 3]
        image2 = Image.open(im2name)
        
        rt = self.raw_data.iloc[idx, 4]
        sigma_or_accuracy = self.raw_data.iloc[idx, 5]
        
        # if you wanted to, you could perturb one of the images. 
        # our final experiments did not do this, though. only some of them 
        # image1 = image1.filter(ImageFilter.GaussianBlur(radius = sigma_or_accuracy))

        if self.transform:
            image1 = self.transform(image1)
            image2 = self.transform(image2)

        sample = {'label1': label1, 'label2': label2, 'image1': image1,
                                            'image2': image2, 'rt': rt, 'acc': sigma_or_accuracy}

        return sample

In [400]:
def RtPsychCrossEntropyLoss(outputs, targets, psych, device):
    num_examples = targets.shape[0]
    batch_size = outputs.shape[0]
    
    print('in the loss')
    print('is nan outputs1', torch.isnan(outputs).any())


    # converting reaction time to penalty
    # 10002 is close to the max penalty time seen in the data
    for idx in range(len(psych)):   
        psych[idx] = abs(10002 - psych[idx])
#         print('the value of psych[idx] is', psych[idx])
    print('is nan psych', torch.isnan(outputs).any())
   
    # adding penalty to each of the output logits 
    for i in range(len(outputs)):
        val = psych[i] / 300
#         print('scaled down psych is', val)
        if np.isnan(val.cpu()):
            val = 0 
            
        outputs[i] += val 
#         print('outputs after val is', outputs[i])
        
    print('is nan outputs2', torch.isnan(outputs).any())

#     print('outputs before log softmax are', outputs)

    outputs = _log_softmax(outputs)
    outputs = outputs[range(batch_size), targets]
    print('is nan outputs3', torch.isnan(outputs).any())

#     print('outputs after log softmax are', outputs)

    foo = - torch.sum(outputs) / num_examples
# #     print('return object is', foo)

    if torch.isnan(foo):
        print('NAN', foo)
        print('len of outputs here', foo.shape)
        print('len of psych is', psych.shape)
        
        1/0
        # but this needs to be on the gpu
#         foo = torch.FloatTensor(0, requires_grad=True).to(device)
        # or we can prevent it all from getting here in the first place
#         print('foo after adjustment hehe', foo)
        
    return foo

def _softmax(x):
    exp_x = torch.exp(x)
    sum_x = torch.sum(exp_x, dim=1, keepdim=True)

    return exp_x/sum_x

def _log_softmax(x):
    return torch.log(_softmax(x))

In [401]:
class MetricCallback(Callback):
    def __init__(self):
        super().__init__()
        self.metrics = []

    def on_validation_epoch_end(self, trainer, pl_module):
        self.metrics.append(trainer.callback_metrics)

In [402]:
# data transforms and loader 
train_transform = T.Compose([
#                 T.RandomCrop(32, padding=4),
                T.Grayscale(num_output_channels=3),
#                 T.RandomHorizontalFlip(),
                T.ToTensor(),
                T.Normalize(mean=[0.9273, 0.9273, 0.9273], std=[0.1775, 0.1775, 0.1775]),

                ])
dataset = OmniglotReactionTimeDataset('sigma_dataset.csv', 
            transforms=train_transform)

test_split = .2
shuffle_dataset = True

dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(test_split * dataset_size))

if shuffle_dataset:
    np.random.seed(3)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

In [403]:
# loader = torch.utils.data.DataLoader(dataset, batch_size=10, 
#                                             sampler=train_sampler)

# dataiter = iter(loader)
# inputs = dataiter.next()

# img = inputs['image1'][0]

# mean, std = img.mean([1,2]), img.std([1,2])
 
# # print mean and std
# print("mean and std before normalize:")
# print("Mean of the image:", mean)
# print("Std of the image:", std)



In [404]:
class Model(LightningModule):
    def __init__(self, **kwargs):
        super(Model, self).__init__()

        self.epoch = 0
        self.learning_rate = args.learning_rate
        self.batch_size = args.batch_size
        self.num_classes = args.num_classes
        self.dataset_name = args.dataset_name
        self.model_name = args.model_name
        self.loss_name = args.loss_fn

        self.default_loss_fn = nn.CrossEntropyLoss()

        # define model - using argparser or someting like tha
        if self.model_name == 'ViT':
            print('use the other script')
            pass
        elif self.model_name == 'VGG':
            self.model = torchvision.models.vgg16(pretrained=True)
            
            self.fc = nn.Linear(4096, self.num_classes) # might need to be 1000 in ...
            # you might need to mod classes here
        elif self.model_name == 'googlenet':
            self.model = torchvision.models.googlenet(pretrained=True)
        elif self.model_name == 'alexnet':
            self.model = torchvision.models.alexnet(pretrained=True)
        else:
            self.model = torchvision.models.resnet50(pretrained=True)
            self.fc = nn.Linear(2048, self.num_classes)
            
    def forward(self, x):
        x = self.model(x)
        x = self.fc(x)
        return x

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(dataset, batch_size=self.batch_size, 
                                            sampler=train_sampler)
    
    def val_dataloader(self):
        return torch.utils.data.DataLoader(dataset, batch_size=self.batch_size,
                                                    sampler=valid_sampler)
    
    def test_dataloader(self):
        pass
    
    def training_step(self, batch, batch_idx):  

        self.model.train()
        self.fc.train()
        
        # main objective here, 
        # train with a different psychophysical one given here. 
        # probably on that old one yeah 
        if self.dataset_name == 'omniglot':     
            image1 = batch['image1']
            image2 = batch['image2']

            label1 = batch['label1']
            label2 = batch['label2']

            if self.loss_name == 'psych-acc':
                psych = batch['acc']
            else: 
                psych = batch['rt']

            # concatenate the batched images
            inputs = torch.cat([image1, image2], dim=0)
            labels = torch.cat([label1, label2], dim=0)
            

            print('in training_step, inputs shape', inputs.shape )
            print('in training_step, inputs max', torch.max(inputs) )
            print('in training_step, inputs min', torch.min(inputs) )


#             print('in training_step, labels shape', labels.shape )

            
            print('in training_step, inputs is nan', torch.isnan(inputs).any())
#             print('in training_step, inputs are', inputs)

            # apply psychophysical annotations to correct images
            psych_tensor = torch.zeros(len(labels))
            j = 0 
            for i in range(len(psych_tensor)):
                if i % 2 == 0: 
                    psych_tensor[i] = psych[j]
                    j += 1
                else: 
                    psych_tensor[i] = psych_tensor[i-1]
            psych_tensor = psych_tensor

#             print('before model outtpus')
#             print('in training_step, inputs shape', inputs.shape)
#             print('in training_step, psych_tensor shape', psych_tensor.shape)

#             print('is nan', torch.isnan(inputs).any())
#             print('is nan', torch.isnan(psych_tensor).any())

                        

            outputs = self.model(inputs).to(self.device)
#             print('in training_step, inputs shape', inputs)
            print('after output infinity check:')
            for name, param in self.model.named_parameters():
                print(name, param.grad)
            1/0
                
                
                
            print('in training_step, outputs is nan', torch.isnan(outputs).any())

            
                
            # so this is our old one already, 
            # but we didn't really use it, no?
            loss = None
            if self.loss_name == 'psych_rt':
                loss = RtPsychCrossEntropyLoss(outputs, labels, psych_tensor, device=self.device)
            else:
                loss = self.default_loss_fn(outputs, labels)
            # calculate accuracy per class
            labels_hat = torch.argmax(outputs, dim=1)
            train_acc = torch.sum(labels.data == labels_hat).item() / (len(labels) * 1.0)

        else:
            # Jin's imagenet-modified dataset
            # which we can't fucking use right now

            # this is a setting, but we probably don't need it 

            # if i in rt_indices:
            #     batch = next(rt_iter)

            # elif i in no_rt_indices:
            #     try:
            #         batch = next(no_rt_iter)
            #     except:
            #         continue

            # print('batch is here:', batch)
            input = batch["imgs"]
            rts = batch["rts"]
            target = batch["labels"]
            input_var = torch.autograd.Variable(input)
            target_var = torch.autograd.Variable(target).long()

            # print('shape of the input is:', input_var.shape)
            # print('shape of the target is:', target.shape)

            # TODO: The model expects the rts and handles them in a custom way 
            # 1 - handle them with our custom loss instead 
            # 2 - later, try them with the MSDensenet, too

            # output, feature, end_time = self.model(input_var)

            
            outputs = self.model(input_var)
#             print('outputs are', outputs)

            if self.loss_name == 'cross_entropy':
                loss = self.default_loss_fn(outputs, target_var)
            else:
                # psych loss
                loss = RtPsychCrossEntropyLoss(outputs, target_var, rts,  device=self.device)
            labels_hat = torch.argmax(outputs, dim=1)
            train_acc = torch.sum(target_var.data == labels_hat).item() / (len(target_var) * 1.0)


        self.log('train_loss', loss)
        self.log('train_acc', train_acc)

#         if torch.isnan(loss):
#             loss = None
        
        return {
            'loss': loss,
            'train_acc': train_acc
        }
    
    def validation_step(self, batch, batch_idx):
        
        if self.dataset_name == "omniglot":
            image1 = batch['image1']
            image2 = batch['image2']

            label1 = batch['label1']
            label2 = batch['label2']

            if self.loss_name == 'psych-acc':
                psych = batch['acc']
            else: 
                psych = batch['rt']

            # concatenate the batched images
            inputs = torch.cat([image1, image2], dim=0)
            labels = torch.cat([label1, label2], dim=0)

            # apply psychophysical annotations to correct images
            psych_tensor = torch.zeros(len(labels))
            j = 0 
            for i in range(len(psych_tensor)):
                if i % 2 == 0: 
                    psych_tensor[i] = psych[j]
                    j += 1
                else: 
                    psych_tensor[i] = psych_tensor[i-1]
            psych_tensor = psych_tensor


            outputs = self.model(inputs)

            loss = None
            if self.loss_name == 'psych_rt':
                loss = RtPsychCrossEntropyLoss(outputs, labels, psych_tensor, device=self.device)
            else:
                loss = self.default_loss_fn(outputs, labels)

            # calculate accuracy per class
            labels_hat = torch.argmax(outputs, dim=1)
            val_acc = torch.sum(labels.data == labels_hat).item() / (len(labels) * 1.0)

        else: 
            # this is a setting, but we probably don't need it 

            # if i in rt_indices:
            #     batch = next(rt_iter)

            # elif i in no_rt_indices:
            #     try:
            #         batch = next(no_rt_iter)
            #     except:
            #         continue

            input = batch["imgs"]
            rts = batch["rts"]
            target = batch["labels"]
            input_var = torch.autograd.Variable(input)
            target_var = torch.autograd.Variable(target).long()

            # print('shape of the input is:', input_var.shape)
            # print('shape of the target is:', target.shape)

            # TODO: The model expects the rts and handles them in a custom way 
            # 1 - handle them with our custom loss instead 
            # 2 - later, try them with the MSDensenet, too

            # output, feature, end_time = self.model(input_var)
            
            # see where are files compare here ...
            outputs = self.model(input_var)
            outputs = outputs[1]
#             print('outputs shape is', outputs.shape)
#             print('outputs are', outputs)

            if self.loss_name == 'cross_entropy':
                loss = self.default_loss_fn(outputs, target_var)
            else:
                # psych loss
                loss = RtPsychCrossEntropyLoss(outputs, target_var, rts, device=self.device)

            labels_hat = torch.argmax(outputs, dim=1)
            val_acc = torch.sum(target_var.data == labels_hat).item() / (len(target_var) * 1.0)

        self.log('val_loss', loss)
        self.log('val_acc', val_acc)

        return {
            'val_loss': loss,
            'val_acc': val_acc
        }
    
    def test_step(self, batch, batch_idx):
        
        if self.dataset_name == "omniglot":
            image1 = batch['image1']
            image2 = batch['image2']

            label1 = batch['label1']
            label2 = batch['label2']

            if self.loss_name == 'psych-acc':
                psych = batch['acc']
            else: 
                psych = batch['rt']

            # concatenate the batched images
            inputs = torch.cat([image1, image2], dim=0)
            labels = torch.cat([label1, label2], dim=0)

            # apply psychophysical annotations to correct images
            psych_tensor = torch.zeros(len(labels))
            j = 0 
            for i in range(len(psych_tensor)):
                if i % 2 == 0: 
                    psych_tensor[i] = psych[j]
                    j += 1
                else: 
                    psych_tensor[i] = psych_tensor[i-1]
            psych_tensor = psych_tensor

            outputs = self.model(inputs)

            loss = None
            if self.loss_name == 'psych_rt':
                loss = RtPsychCrossEntropyLoss(outputs, labels, psych_tensor, device=self.device)
            else:
                loss = self.default_loss_fn(outputs, labels)

            # calculate accuracy per class
            labels_hat = torch.argmax(outputs, dim=1)
            val_acc = torch.sum(labels.data == labels_hat).item() / (len(labels) * 1.0)
            
        else: 
            # this is a setting, but we probably don't need it 

            # if i in rt_indices:
            #     batch = next(rt_iter)

            # elif i in no_rt_indices:
            #     try:
            #         batch = next(no_rt_iter)
            #     except:
            #         continue

            input = batch["imgs"]
            rts = batch["rts"]
            target = batch["labels"]
            input_var = torch.autograd.Variable(input)
            target_var = torch.autograd.Variable(target).long()

            # print('shape of the input is:', input_var.shape)
            # print('shape of the target is:', target.shape)

            # TODO: The model expects the rts and handles them in a custom way 
            # 1 - handle them with our custom loss instead 
            # 2 - later, try them with the MSDensenet, too

            # output, feature, end_time = self.model(input_var)
            
            # see where are files compare here ..

            
            outputs = self.model(input_var)
            outputs = outputs[1]
#             print('outputs shape is', outputs.shape)
#             print('outputs are', outputs)

            
            if self.loss_name == 'cross_entropy':
                loss = self.default_loss_fn(outputs, target_var)
            else:
                # psych loss
                loss = RtPsychCrossEntropyLoss(outputs, target_var, rts, device=self.device)


            labels_hat = torch.argmax(outputs, dim=1)
            val_acc = torch.sum(target_var.data == labels_hat).item() / (len(target_var) * 1.0)

        self.log('test_loss', loss)
        self.log('test_acc', val_acc)

        return {
            'test_loss': loss,
            'test_acc': val_acc
        }        

In [405]:
# need a datamodule for this stuff right here, I think 
# nah, just do a dataset and a dataloader for each class, as you do in your main code 

# dataset and dataloaders




In [406]:
class Args:
    num_epochs = 25
    batch_size = 64
    num_classes = 100
    learning_rate = 0.0001
    loss_fn = 'psych_rt'
    model_name = 'resnet'
    dataset_name = 'omniglot'
    log = False
args=Args()

#TODO: just add the seeds again at some point
path = '/afa/crc.nd.edu/user/j/jdulay/.cache/'
if os.path.isdir(path):
    os.rmdir(path)
seed_everything(3, workers=True)

metrics_callback = MetricCallback()

wandb_logger = None
if args.log:
    logger_name = "01_train_omniglot_psych".format(args.model_name, args.dataset_name, 3)
    wandb_logger = WandbLogger(name=logger_name,project="train_omniglot_psych")

trainer = pl.Trainer(
    max_epochs=args.num_epochs,
    devices=1, 
    accelerator='gpu',            
#     strategy='ddp',
    auto_select_gpus=True,
    callbacks=[metrics_callback],
    logger=wandb_logger,
    progress_bar_refresh_rate=1000,
#     limit_train_batches=0,
    limit_val_batches=0
) 

model_ft = Model()

# data_module = DataModule(data_dir=args.dataset_name, batch_size=args.batch_size)

trainer.fit(model_ft)
trainer.save_checkpoint("omniglot_psych.ckpt")

# trainer.test(model_ft, data_module, ckpt_path=None)


Global seed set to 3
Auto select gpus: [0]
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name            | Type             | Params
-----------------------------------------------------
0 | default_loss_fn | CrossEntropyLoss | 0     
1 | model           | ResNet           | 25.6 M
2 | fc              | Linear           | 204 K 
-----------------------------------------------------
25.8 M    Trainable params
0         Non-trainable params
25.8 M    Total params
103.048   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

in training_step, inputs shape torch.Size([128, 3, 28, 28])
in training_step, inputs max tensor(0.4096, device='cuda:0')
in training_step, inputs min tensor(-5.2242, device='cuda:0')
in training_step, inputs is nan tensor(False, device='cuda:0')
after output infinity check:
conv1.weight None
bn1.weight None
bn1.bias None
layer1.0.conv1.weight None
layer1.0.bn1.weight None
layer1.0.bn1.bias None
layer1.0.conv2.weight None
layer1.0.bn2.weight None
layer1.0.bn2.bias None
layer1.0.conv3.weight None
layer1.0.bn3.weight None
layer1.0.bn3.bias None
layer1.0.downsample.0.weight None
layer1.0.downsample.1.weight None
layer1.0.downsample.1.bias None
layer1.1.conv1.weight None
layer1.1.bn1.weight None
layer1.1.bn1.bias None
layer1.1.conv2.weight None
layer1.1.bn2.weight None
layer1.1.bn2.bias None
layer1.1.conv3.weight None
layer1.1.bn3.weight None
layer1.1.bn3.bias None
layer1.2.conv1.weight None
layer1.2.bn1.weight None
layer1.2.bn1.bias None
layer1.2.conv2.weight None
layer1.2.bn2.weight None


ZeroDivisionError: division by zero